# Réseaux Recursifs (RNNs) et Modèles de Langage
Pour ce TP nous allons explorer les RNNs et notamment les LSTMs. Nous allons essayer d'utiliser un RNN pour apprendre des séquences des mots (un texte) et ensuite générer de nouvelles séquences. 

Néanmoins, avant cela nous allons examiner la structure basique d'un LSTM en utilisant une entrée aléatoire. Même si plus tard nous allons avoir du texte en entrée, les LSTMs fonctionnent avec des nombres. Nous allons voir plus tard comment passer du texte aux tenseurs. Pour l'instant voici un tenseur 3x8 aléatoire. 




In [1]:
import torch
from torch import nn
import pandas as pd
from collections import Counter


In [2]:
#entrée
x = torch.randint(1, 100, (3, 8))
print(x)

# torch.randint(low=0, high, size, \*, generator=None, out=None, dtype=None,
#   layout=torch.strided, device=None, requires_grad=False) → Tensor


tensor([[66, 10, 46, 24, 59, 19, 32, 82],
        [40, 21, 39, 28, 54, 12, 47, 59],
        [42, 64, 81, 28, 21,  5, 64, 21]])


Nous allons passer cet entrée aléatoire à une couche embedding, parce que les embeddings des mots peuvent mieux  representer le contexte et sont plus efficaces que les representations one-hot. 

Pour Pytorch nous avons besoin d'utiliser `nn.Embedding` afin de créer cette couche, qui prend en entrée la taille du vocabulaire et la longueur de vecteur de mot souhaitée. Vous pouvez éventuellement fournir un index de padding, pour indiquer l'index de l'élément de padding dans la matrice qui va représenter une phrase. Le padding sert à mettre ensemble plusieurs phrases dans un minibatch pour les mettre toutes à la même longueur.


Dans l'exemple suivant, notre vocabulaire se compose de 100 mots, incluant l'élément spécial du padding, pour lequel on a choisi de donner l'indice 0.

Remarque : dans cet exemple, le padding ne sert pas...

Que représente `x` dans notre contexte NLP ? Quelle est la taille des tenseurs issus de `model1` de la cellule suivante ?

- _x représente une mise en forme du dictionaire de mots. Le réseau trouve une représentation autour que one-hot._ 

In [3]:
model1 = nn.Embedding(100, 7, padding_idx=0)
out1 = model1(x)

Nous passons la sortie de la couche embedding dans une couche LSTM qui prend en entrée la longeur du vecteur representant le mot, la longueur de la couche cachée, et le nombre des couches. La couche LSTM sort trois choses, à quoi correspondent chacune d'entre elles ? Quelles sont leurs tailles ?

- **out** contient la sortie de la denrnière couche du LSTM pour chaque epoch de taille (longeur sequence, 1, taille du batch, nombre d'état de sortie).
- **h_n** contient l'état caché final de chaque élément de la séquence de taille (nombre de couches, taille du batch, nombre de couches cachées).
- **c_n** contient la mémoire cell finale de chaque élélment de la séquence de taille (nombre de couche, taille du batch, nombre de couches cahcées).

_source :_ https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

In [4]:
model2 = nn.LSTM(input_size=7, hidden_size=5, num_layers=1, batch_first=True)

out, (ht, ct) = model2(out1)


Passons maintaiannt au modèle de langage. Pour les données nous allons utiliser un ensemble de blagues recueillis sur Reddit, inspiré par un tutoriel de Domas Bitvinskas.   


## Le modèle 
Voici un premier modèle utilisant trois couches LSTM.

In [5]:
class Model(nn.Module):

    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3
        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))


C'est un model LSTM avec Pytorch assez standard. Comme expliqué dans l'introduction, le but de la couche `Embedding` est de convertir les mots (leur indice dans un dictionnaire) en vecteurs. La fonction `init_state` est appelée au début de chaque époque.  

## Données 
Téléchargons les données.




In [6]:
!wget https://raw.githubusercontent.com/amoudgl/short-jokes-dataset/master/data/reddit-cleanjokes.csv

--2023-02-13 12:49:26--  https://raw.githubusercontent.com/amoudgl/short-jokes-dataset/master/data/reddit-cleanjokes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141847 (139K) [text/plain]
Saving to: ‘reddit-cleanjokes.csv’

reddit-cleanjokes.c 100%[===================>] 138.52K  --.-KB/s    in 0.002s  

2023-02-13 12:49:26 (60.4 MB/s) - ‘reddit-cleanjokes.csv’ saved [141847/141847]




Il s'agit d'un fichier tsv de la forme `ID,Joke` ou `ID` signifie simplement l'identifiant de la « blague » et `Joke` le texte. Afin de pouvoir traiter les données nous aurons besoin d'utiliser la class `Dataset`


In [7]:
class Dataset(torch.utils.data.Dataset):
  
    def __init__(
        self,
        sequence_length,
    ):
        self.sequence_length = sequence_length
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}
        self.words_indexes = [self.word_to_index[w] for w in self.words]
    
    def load_words(self):
        train_df = pd.read_csv('reddit-cleanjokes.csv')
        text = train_df['Joke'].str.cat(sep=' ')
        return text.split(' ')
    
    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)
    
    def __len__(self):
        return len(self.words_indexes) - self.sequence_length
    
    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

Cette classe hérite de la classe `torch.utils.data.Dataset`. En plus de `__init__`,  il est nécessaire de définir deux fonctions : `__len__` et `__getitem__`. La première retourne la taille de notre ensemble des données alors que la deuxième implémente l'indexation afin que `dataset[i]` puisse être utilisé pour retourner le *i*-ème élément. Vous pouvez avoir plus de détails [ici](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class). 

La fonction `load_words` charge le dataset. Le but est de trouver tous les mots afin de définir la taille du vocabulaire du réseau mais également la taille de l'embedding. Deux autres fonctions `index_to_word` et `word_to_index` convertissent les mots en index et vice versa. 

# Entrainement 

Nous allons définir une fonction `train` pour entraîner notre RNN. 

In [8]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader

batch_size = 256


def train(dataset, model, max_epochs, sequence_length):
    model.train()
    dataloader = DataLoader(dataset, batch_size)
    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.01)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(sequence_length)
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)
            state_h = state_h.detach()
            state_c = state_c.detach()
            loss.backward()
            optimizer.step()
            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

Comme vous pouvez constater la fonction train charge d'abord les données, définit comme fonction de perte Cross Entropy Loss ainsi que SGD comme optimizer et ensuite appelle le modèle pour `max_epochs`. 

# Prédictions 

Une fois que nous avons entraîné notre modèle nous pouvons ensuite faire des prédictions, en donnant une séquence des mots en entrée.   Voici une fonction nous permettant de prédire les `next_words` suivant à partir d'un `text`. 

In [9]:
def predict(dataset, model, text, next_words=100):
    model.eval()
    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))
    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])
    return words

Nous sommes prêtes et prêts maintenant à entraîner notre modèle, ci-dessous un morceau du code qui nous permettra de le faire sur l'ensemble des données.

In [10]:
sequence_length = 4
max_epochs = 10

dataset = Dataset(sequence_length)
model = Model(dataset)
print(model)
train(dataset, model, max_epochs, sequence_length)
print(predict(dataset, model, text='One day I shot an elephant in my suit. I have no idea how he got into it.'))

Model(
  (embedding): Embedding(6925, 128)
  (lstm): LSTM(128, 128, num_layers=3, dropout=0.2)
  (fc): Linear(in_features=128, out_features=6925, bias=True)
)
{'epoch': 0, 'batch': 0, 'loss': 8.835528373718262}
{'epoch': 0, 'batch': 1, 'loss': 8.826991081237793}
{'epoch': 0, 'batch': 2, 'loss': 8.824238777160645}
{'epoch': 0, 'batch': 3, 'loss': 8.82489013671875}
{'epoch': 0, 'batch': 4, 'loss': 8.81036376953125}
{'epoch': 0, 'batch': 5, 'loss': 8.797056198120117}
{'epoch': 0, 'batch': 6, 'loss': 8.792482376098633}
{'epoch': 0, 'batch': 7, 'loss': 8.772226333618164}
{'epoch': 0, 'batch': 8, 'loss': 8.73105525970459}
{'epoch': 0, 'batch': 9, 'loss': 8.67509651184082}
{'epoch': 0, 'batch': 10, 'loss': 8.594453811645508}
{'epoch': 0, 'batch': 11, 'loss': 8.471065521240234}
{'epoch': 0, 'batch': 12, 'loss': 8.398972511291504}
{'epoch': 0, 'batch': 13, 'loss': 8.289005279541016}
{'epoch': 0, 'batch': 14, 'loss': 8.08060073852539}
{'epoch': 0, 'batch': 15, 'loss': 8.015557289123535}
{'epoch'



# Exercises 

1.  Nous allons essayer d'améliorer les résultats de notre réseau récurrent. Essayez de changer la taille de la représentation cachée (hidden representation size). Les résultats sont-ils meilleurs ?

2. Essayez d'expérimenter avec une couche récurrente supplémentaire. Avez-vous obtenu de meilleurs résultats ? 

3. Expérimentez également avec le dropout rate et essayez de voir si vos résultats sont meilleurs. 

4. Transformez le LSTM en un LSTM bidirectionnel 


5. La fonction `predict` choisit aléatoirement, selon la distribution des probabilités, un mot. Essayez de faire la même chose, mais en se limitant sur les 4 mots les plus probables. Autrement dit, utilisez une distribution de probabilités uniforme sur ces 4 mots. Les résultats se sont-ils améliorés ? Que doit-on faire pour les améliorer encore ?



In [18]:
def predict(dataset, model, text, next_words=100):
    model.eval()
    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))
    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach()#.numpy()
        # saving the indices of topk which records 4 words with best prob
        best_prob = torch.topk(p, 4).indices 
        word_index = np.random.choice(best_prob)
        words.append(dataset.index_to_word[word_index])
    return words

print(predict(dataset, model, text='One day I shot an elephant in my suit. I have no idea how he got into it.', next_words = 2))

['One', 'day', 'I', 'shot', 'an', 'elephant', 'in', 'my', 'suit.', 'I', 'have', 'no', 'idea', 'how', 'he', 'got', 'into', 'it.', 'and', 'the']


6. Vous pouvez télécharger un autre jeu de données, des recettes de cuisine. Tester le modèle sur ces nouvelles données.

In [11]:
!wget https://www.irit.fr/~Thomas.Pellegrini/ens/M1ML1/sents_recipes.txt

--2023-02-13 12:58:26--  https://www.irit.fr/~Thomas.Pellegrini/ens/M1ML1/sents_recipes.txt
Resolving www.irit.fr (www.irit.fr)... 141.115.28.2
Connecting to www.irit.fr (www.irit.fr)|141.115.28.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘sents_recipes.txt’

sents_recipes.txt       [   <=>              ] 574.38K   581KB/s    in 1.0s    

2023-02-13 12:58:29 (581 KB/s) - ‘sents_recipes.txt’ saved [588160]



7. Voici un troisième jeu de données : une liste de noms de villes françaises. Cette fois-ci, l'idée est de travailler au niveau des caractères plutôt que des mots. Modifier le notebook pour gérer les caractères. La taille de la séquence à modéliser peut être plus grande que pour les mots. 


In [12]:
!wget https://www.irit.fr/~Thomas.Pellegrini/ens/M1ML1/communes_france.txt

--2023-02-13 12:58:29--  https://www.irit.fr/~Thomas.Pellegrini/ens/M1ML1/communes_france.txt
Resolving www.irit.fr (www.irit.fr)... 141.115.28.2
Connecting to www.irit.fr (www.irit.fr)|141.115.28.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘communes_france.txt’

communes_france.txt     [   <=>              ] 434.20K   556KB/s    in 0.8s    

2023-02-13 12:58:30 (556 KB/s) - ‘communes_france.txt’ saved [444616]

